In [20]:
from neural_network import *

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

In [2]:
input_size1 = 5
output_size1 = 2
input_size2 = 3
output_size2 = 2

filename = 'model.pt'

NN = Neural_network(input_size1,output_size1,input_size2,output_size2)

NN.load(filename)
NN.eval()

Neural_network(
  (net1): Sequential(
    (0): Linear(in_features=5, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.0, inplace=False)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.0, inplace=False)
    (11): Linear(in_features=50, out_features=2, bias=True)
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.0, inplace=False)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.0, inplace=False)
    (11): Linear(in_features=50, 

In [3]:
file_path = "../Data/01_Inicial_v8.xlsx"

D = Data_loader(file_path)

In [16]:
inputs = np.zeros((0,input_size1+input_size2))
targets = np.zeros((0,output_size1+output_size2))
preds = np.zeros((0,output_size1+output_size2))

for X, Y_target in D.get_batch2(batch_size=10):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2,Y1_target,Y2_target = D.denormalize_values(X1=X1,Y1=Y1_target,X2=X2,Y2=Y2_target)
    _,_,Y1_pred,Y2_pred = D.denormalize_values(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

n_train = inputs.shape[0]

for X, Y_target in D.get_batch2(batch_size=10,validation=True):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2,Y1_target,Y2_target = D.denormalize_values(X1=X1,Y1=Y1_target,X2=X2,Y2=Y2_target)
    _,_,Y1_pred,Y2_pred = D.denormalize_values(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

train = np.zeros((inputs.shape[0],1))
train[:n_train,0] = 1



In [26]:

input_names = D.input_columns_DDO + D.input_columns_RBDO
output_names = D.output_columns_DDO + D.output_columns_RBDO
output_names_target = [f'{c}_target' for c in output_names]
output_names_pred = [f'{c}_pred' for c in output_names]

df = pd.DataFrame(np.hstack((inputs,targets,preds,train)), columns=input_names+output_names_target+output_names_pred+['train'])

df.to_csv('output.csv', sep=',', index=False)

In [22]:
df

,c,phi,gamma,HV,H,c_cov,phi_cov,gamma_cov,V1_target,FS_target,V2_target,beta_target,V1_pred,FS_pred,V2_pred,beta_pred,train
0,18.0,22.0,18.0,1.5,10.0,16.0,29.0,7.0,78.045193,1.627,82.880308,2.574978,78.906226,1.682537,80.315444,2.365796,1.0
1,20.0,25.0,18.0,1.0,10.0,50.0,30.0,11.0,41.291581,1.510,38.168736,1.042700,46.207994,1.524529,63.544316,0.995996,1.0
2,8.0,31.0,17.0,1.0,30.0,24.0,21.0,9.0,225.783856,1.060,205.486231,0.164800,201.451288,1.123561,241.832445,0.369838,1.0
3,12.0,32.0,19.0,2.0,20.0,20.0,22.0,10.0,397.043190,1.940,438.498140,3.178984,360.314467,1.984212,396.929363,3.072180,1.0
4,12.0,22.0,19.0,2.0,30.0,39.0,10.0,11.0,797.511202,1.273,404.002060,1.859900,830.821150,1.294503,489.286858,1.626490,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,30.0,30.0,20.0,1.5,20.0,50.0,30.0,11.0,235.159044,1.927,149.322275,1.776600,248.470969,1.935550,118.749210,1.652247,0.0
265,8.0,33.0,15.0,2.0,10.0,12.0,16.0,9.0,116.303235,2.028,116.303240,5.153901,101.661847,2.009663,171.836641,4.942799,0.0
266,28.0,23.0,18.0,1.0,20.0,37.0,12.0,8.0,191.745442,1.327,165.300745,1.283400,176.084055,1.333427,171.342349,1.036100,0.0
267,18.0,21.0,17.0,1.5,20.0,15.0,18.0,11.0,235.159044,1.308,276.248612,1.827300,230.556800,1.320520,307.134951,1.411508,0.0
